<a href="https://colab.research.google.com/github/avilaJorge/CSE253_FinalProject/blob/master/DeepFakeDetectionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%run DeepFake_Dataloader.py
%run utils.py

In [2]:
# imports pytorch
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

# Numpy, Matplotlib, and PIL
import numpy as np
import matplotlib.pyplot as plt

# general imports
import pprint

# My imports
from DeepFake_Dataloader import get_dataloaders
from models import LinearRegression
from utils import RunningAverage

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
trn_dl, val_dl, tst_dl = get_dataloaders()

('Classes: '
 "['.ipynb_checkpoints', "
 "'100KFake_10K', "
 "'Flickr-Faces-HQ_10K', "
 "'celebA-HQ_10K', "
 "'thispersondoesntexists_10K']")
('Indices: '
 "{'.ipynb_checkpoints': "
 '0, '
 "'100KFake_10K': "
 '1, '
 "'Flickr-Faces-HQ_10K': "
 '2, '
 "'celebA-HQ_10K': "
 '3, '
 "'thispersondoesntexists_10K': "
 '4}')


In [4]:
# img2 = Image.open('./Frequency/Faces-HQ/celebA-HQ_10K/0.jpg')
# # display(img2)
# img2 = img2.convert('L')

# mag_spectrum = np_magnitude_spectrum(img2)
# rad2 = np_radial_profile(mag_spectrum, center=(mag_spectrum.shape[0]/2, mag_spectrum.shape[1]/2))
# plt.plot(rad2)
# plt.show()

# rad_p, t = next(iter(trn_dl))
# rad_p = rad_p.squeeze(0)
# # t_img = torch.from_numpy(np.asarray(img2)).squeeze(0).float()
# # ms = magnitude_spectrum(t_img)
# # rad2 = radial_profile(ms, center=(ms.shape[0]/2, ms.shape[1]/2))
# plt.plot(rad_p.cpu())
# plt.show()

In [5]:
s_epoch      = 1
num_epochs   = 100
log_step     = 10
lr_rate      = 1e-5
best_loss    = float('inf')
prev_loss    = float('inf')
loss_inc_cnt = 0
stop_early   = False
load_model   = False
model_name   = 'LogisticRegressionModel'

lr_model = LinearRegression(725, 250, 1).to(device)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(lr_model.parameters(), lr=lr_rate)

if load_model:
    lr_model.load_state_dict(torch.load(''))

In [6]:
def train(model, dataloader, validation_loader):
    
    # Train the models
    total_step = len(dataloader)
    for epoch in range(s_epoch, num_epochs):
        ts = time.time()
        print_info("----- Starting Training Epoch lr=%.8f best_loss=%f ------\n" 
                   % (lr_rate, best_loss))
        for i, (x, y) in enumerate(dataloader):
            optimizer.zero_grad()

            # Move to GPU
            x = x.to(device)
            y = y.to(device).float().unsqueeze(1)

            # Forward, backward and optimize
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            loss=loss.item()

            # Print log info
            if i % log_step == 0:
                print_info('"Epoch": [{}/{}], "Step": [{}/{}], "Loss": {:.4f} \n'
                      .format(epoch, num_epochs, i, total_step, loss))
                
        print_info("Finished Training. Time elapsed %f \n" % (time.time() - ts))

        loss = evaluate(model, epoch, dataloader, validation=False)
        print_info('"Training Epoch Done": "Epoch": [{}/{}], "Loss": {:.4f} \n'
                      .format(epoch, num_epochs, loss)) 
        
        loss = evaluate(model, epoch, validation_loader) 
        print_info('"Validation Epoch Done": "Epoch": [{}/{}], "Loss": {:.4f} \n'
                      .format(epoch, num_epochs, loss))

def evaluate(model, epoch, data_loader, validation=True):

    global best_loss
    global prev_loss
    global loss_inc_cnt
    global stop_early
    
    with torch.no_grad():
        losses = RunningAverage()
        ts = time.time()
        print_info("----- Starting Evaluation ------\n")
        total_step = len(data_loader)
        
        for i, (x, y) in enumerate(data_loader):

            # Move to GPU
            x = x.to(device)
            y = y.to(device).float().unsqueeze(1)

            # Evaluate
            pred = model(x)
            losses.update(criterion(pred, y).item())

        print_info("Finished evaluation. Time elapsed %f \n" % (time.time() - ts))
        loss = losses.value

        if validation:
            if best_loss > loss:
                best_loss = loss
                print_info('"Best Loss": ' + str(best_loss) + '\n')
                save_model(model, 'Best-' + model_name, dt, path)

            loss_inc_cnt = loss_inc_cnt + 1 if prev_loss < loss else 0
            if loss_inc_cnt > 2: stop_early = True
            save_model(model, model_name, dt, path)
            print_info("Validation Loss has gone up %d times.\n" % (loss_inc_cnt))
            prev_loss = loss
        
        return loss


In [7]:
train(lr_model, trn_dl, val_dl)

----- Starting Training Epoch lr=0.00001000 best_loss=inf ------

"Epoch": [1/100], "Step": [0/188], "Loss": 0.7019 



KeyboardInterrupt: 